<a href="https://colab.research.google.com/github/Romaindujardin/SPOT/blob/main/Pipeline_SPOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SPOT

SPOT est un mini-système de projet de cours visant à simuler un appel automatique en classe via reconnaissance faciale.

## 1. Importation & installation des bibliothèques

In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output

# OpenCV contrib (LBPH) — on force une install propre (évite les conflits opencv-python/opencv-contrib)
%pip -q uninstall -y opencv-python opencv-python-headless opencv-contrib-python opencv-contrib-python-headless
%pip -q install --upgrade pillow-heif opencv-contrib-python-headless

import cv2
print("OpenCV version:", cv2.__version__)
print("cv2.face disponible:", hasattr(cv2, "face") and hasattr(cv2.face, "LBPHFaceRecognizer_create"))

if not (hasattr(cv2, "face") and hasattr(cv2.face, "LBPHFaceRecognizer_create")):
    raise RuntimeError(
        "cv2.face est toujours indisponible."
    )


### Définition de la structure classes et professeurs

```
Eleve 1/
        Photo1.png
        Photo2.png
Eleve 2/
        Photo1.png
        Photo2.png

```


Dans notre scénario, quand le prof arrive en cours, il scan sa carte d'école qui va ensuite lui permettre de choisir sa classe.

Dans le dataset chaque professeur a plusieurs classes, chaque classe a plusieurs élèves et chaque élève a ses images enrigistrés pour entrainer le modèle.

Ici on doit sélectionner le professeur et la classe pour avoir les bons élèves durant le cours et cliquer sur charger la classe.

In [ ]:
# On initialise la variable ici pour qu'elle existe même avant le clic
global ELEVES_ATTENDUS
ELEVES_ATTENDUS = []

# Structure
ecole_structure = {
    "Professeur A": {
        "IA": ["romain", "laurence", "chantal", "sophie", "grégory", "nathan"],
        "Cyber": ["Eleve_Cy_1", "Eleve_Cy_2", "Eleve_Cy_3", "Eleve_Cy_4", "Eleve_Cy_5"]
    },
    "Professeur B": {
        "IA": ["Eleve_IA_1", "Eleve_IA_2", "Eleve_IA_3", "Eleve_IA_4", "Eleve_IA_5"],
        "Cyber": ["Eleve_Cy_1", "Eleve_Cy_2", "Eleve_Cy_3", "Eleve_Cy_4", "Eleve_Cy_5"]
    }
}

print("\n--- SÉLECTION DU COURS ---")
prof_dropdown = widgets.Dropdown(options=ecole_structure.keys(), description='Prof:')
classe_dropdown = widgets.Dropdown(description='Classe:')
btn_select = widgets.Button(description="Charger la classe", button_style='success')
output = widgets.Output()

# Mise à jour des listes déroulantes
def update_classes(change):
    classe_dropdown.options = ecole_structure[change.new].keys()
prof_dropdown.observe(update_classes, names='value')
classe_dropdown.options = ecole_structure[prof_dropdown.value].keys()

def on_button_click(b):
    global ELEVES_ATTENDUS
    with output:
        clear_output()
        prof = prof_dropdown.value
        classe = classe_dropdown.value

        # On remplit la variable globale
        ELEVES_ATTENDUS = ecole_structure[prof][classe]

        print(f"Cours : {prof} - Classe {classe}")
        print(f"Élèves chargés : {ELEVES_ATTENDUS}")
        print("-" * 30)

        # Vérif rapide (dataset WEBCAM)
        DATASET_ROOT = "/content/dataset_crop"
        if os.path.exists(DATASET_ROOT):
            missings = [e for e in ELEVES_ATTENDUS if not os.path.exists(os.path.join(DATASET_ROOT, e))]
            if not missings:
                print("Tous les dossiers élèves sont présents (dataset webcam).")
            else:
                print(f"Dossiers manquants (à créer après) : {missings}")
        else:
            print("INFO: dataset webcam pas encore créé. Utilise les boutons WEBCAM juste en dessous.")

btn_select.on_click(on_button_click)
display(prof_dropdown, classe_dropdown, btn_select, output)

(ne pas oublier de faire son choix et d'appuyer sur charger la classe avant de passer à l'étape d'après, sinon rien n'est défini)

## 2. Création du dataset via WEBCAM

A présent au lieu de créer en amont le dataset, on le crée directement dans colab pour avoir les memes conditions de webcam.

On applique le pré-traitement lors de la création du dataset, on sauvegarde tout en png et on applique un crop de visage pour conserver uniquement le visage et plus les arrières plans pour éviter que le modèle apprenne un arrière plan = un élève.

In [ ]:
import os
import time
import base64
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display, Javascript, clear_output
from google.colab.output import eval_js

# 1) Vérifs
if 'ELEVES_ATTENDUS' not in globals() or not ELEVES_ATTENDUS:
    print("Aucun élève chargé. il Faut d'abord: sélectionner Prof + Classe puis 'Charger la classe'.")
else:
    print(f"Élèves chargés: {ELEVES_ATTENDUS}")

DATASET_WEBCAM_ROOT = "/content/dataset_crop"
os.makedirs(DATASET_WEBCAM_ROOT, exist_ok=True)
print("Dataset webcam:", DATASET_WEBCAM_ROOT)

# 2) Caméra JS
js_camera_dataset = """
    if (!document.getElementById('videoElement')) {
        var video = document.createElement('video');
        video.id = 'videoElement';
        video.style.display = 'none';
        document.body.appendChild(video);
        var canvas = document.createElement('canvas');
        canvas.id = 'canvasElement';
        canvas.style.display = 'none';
        document.body.appendChild(canvas);
    }
    async function stream_frame() {
        var video = document.getElementById('videoElement');
        var canvas = document.getElementById('canvasElement');
        if (video.paused) {
            var stream = await navigator.mediaDevices.getUserMedia({
              video: {
                width: {ideal: 1280},
                height: {ideal: 720},
                facingMode: 'user'
              }
            });
            video.srcObject = stream;
            await video.play();
        }
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        return canvas.toDataURL('image/jpeg', 0.92);
    }
"""
display(Javascript(js_camera_dataset))

def js_to_image(js_reply):
    image_bytes = base64.b64decode(js_reply.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

# IMPORTANT: on sauvegarde directement ce que LBPH consomme (200x200 GRAY + CLAHE)
# => training/live parfaitement alignés, distances beaucoup plus basses.
clahe_ds = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

def capture_one_face_ready():
    """Capture 1 frame, détecte le plus grand visage, retourne face_ready (GRAY 200x200 + CLAHE)."""
    js_reply = eval_js('stream_frame()')
    frame = js_to_image(js_reply)
    if frame is None:
        return None, "frame None"

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(140, 140))
    if len(faces) == 0:
        return None, "pas de visage détecté (approche-toi / lumière)"

    (x, y, w, h) = max(faces, key=lambda f: f[2] * f[3])
    margin = max(10, int(0.15 * min(w, h)))
    x0 = max(0, x - margin)
    y0 = max(0, y - margin)
    x1 = min(gray.shape[1], x + w + margin)
    y1 = min(gray.shape[0], y + h + margin)

    face_roi = gray[y0:y1, x0:x1]
    if face_roi.shape[0] < 140 or face_roi.shape[1] < 140:
        return None, "visage trop petit (rapproche-toi)"

    face_resized = cv2.resize(face_roi, (200, 200))
    face_ready = clahe_ds.apply(face_resized)
    return face_ready, f"ok {w}x{h}"

def save_crops_for_student(student: str, n: int, out_widget: widgets.Output):
    student = student.strip()
    os.makedirs(os.path.join(DATASET_WEBCAM_ROOT, student), exist_ok=True)

    saved = 0
    tries = 0
    with out_widget:
        clear_output()
        print(f"Capture {n} images pour: {student}")
        print("Conseil: regarde caméra, puis tourne légèrement gauche/droite, varie expression.")

    while saved < n and tries < n * 6:
        tries += 1
        face_ready, msg = capture_one_face_ready()
        if face_ready is None:
            continue

        ts = int(time.time() * 1000)
        # On sauvegarde en PNG (pas de perte JPEG) + déjà prétraité pour LBPH
        path = os.path.join(DATASET_WEBCAM_ROOT, student, f"webcam_{ts}.png")
        cv2.imwrite(path, face_ready)
        saved += 1
        time.sleep(0.08)

        if saved % 5 == 0:
            with out_widget:
                clear_output()
                print(f"Capture {n} images pour: {student}")
                print(f"Progress: {saved}/{n} (tentatives={tries})")

    with out_widget:
        clear_output()
        print(f"Terminé: {saved}/{n} pour {student} (tentatives={tries})")
        print(f"Dossier: {os.path.join(DATASET_WEBCAM_ROOT, student)}")
        print("Ensuite: relance la cellule d'entraînement LBPH.")

# 3) UI: un bouton par élève + slider nb
n_slider = widgets.IntSlider(value=30, min=10, max=80, step=5, description='Nb:', continuous_update=False)
out = widgets.Output()

btns = []
if 'ELEVES_ATTENDUS' in globals() and ELEVES_ATTENDUS:
    for eleve in ELEVES_ATTENDUS:
        b = widgets.Button(description=f"{eleve}", button_style='info')
        def make_handler(name):
            def _h(_):
                save_crops_for_student(name, int(n_slider.value), out)
            return _h
        b.on_click(make_handler(eleve))
        btns.append(b)

ui = widgets.VBox([
    widgets.HTML("<h3>Création dataset WEBCAM (1 élève à la fois)</h3>"),
    n_slider,
    widgets.HBox(btns) if btns else widgets.HTML("<b>Charge d'abord une classe.</b>"),
    out
])
display(ui)



### Vérification du dataset

### Installation des bibliothèques disponibles

In [ ]:
import os
from pillow_heif import register_heif_opener

register_heif_opener()

# Dataset final utilisé partout
LOCAL_CROP_DIR = "/content/dataset_crop"

print("dataset =", LOCAL_CROP_DIR)



## 3. Entrainement du modèle sur le dataset

Pour l'entrainement, on applique du data-augmentation pour augmenter notre dataset.

On change quelques petits détails pour renforcer le modèle, un peu de flou ou autre.

In [ ]:
import cv2
import os
import numpy as np

# Chemin vers le dataset
DATASET_ROOT = "/content/dataset_crop"

print("Démarrage de l'entraînement LBPH (Direct en mémoire)...")
print(f"Dataset utilisé pour l'entraînement : {DATASET_ROOT}")

# --- 1. PARAMÈTRES ROBUSTES (Rayon et Grille augmentés) ---
# Radius=3 : Regarde des pixels plus éloignés (mieux pour les visages un peu flous)
# Neighbors=12 : Prend plus de voisins en compte
# Grid=10x10 : Découpe le visage en 100 petites zones (plus précis)
recognizer = cv2.face.LBPHFaceRecognizer_create(
    radius=3,
    neighbors=12,
    grid_x=10,
    grid_y=10
)

# --- 2. FONCTION D'AUGMENTATION ---
# En mode WEBCAM, les images sont déjà prétraitées (200x200 GRAY + CLAHE).
# On évite les augmentations "luminosité" qui dégradent souvent LBPH.
def augment(img):
    """Génère 2 variations (original + léger flou)."""
    return [img, cv2.GaussianBlur(img, (3, 3), 0)]

# --- 3. PRÉPARATION DU DATASET ---

faces = []
labels = []
label_map = {}
current_label = 0

if os.path.exists(DATASET_ROOT):
    # On trie pour garantir l'ordre
    for eleve in sorted(os.listdir(DATASET_ROOT)):
        eleve_path = os.path.join(DATASET_ROOT, eleve)

        if not os.path.isdir(eleve_path): continue

        label_map[current_label] = eleve
        print(f" - Traitement de : {eleve} (ID={current_label})")

        image_count = 0
        for file in os.listdir(eleve_path):
            if file.lower().endswith(('jpg', 'jpeg', 'png')):
                img_path = os.path.join(eleve_path, file)

                # Lecture
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None: continue

                # A. REDIMENSIONNEMENT (sécurité)
                if img.shape != (200, 200):
                    img = cv2.resize(img, (200, 200))

                # IMPORTANT: pas de CLAHE ici (déjà appliqué lors de la capture WEBCAM)

                # B. AUGMENTATION
                augmented_images = augment(img)

                # D. AJOUT AU DATASET
                for aug_img in augmented_images:
                    faces.append(aug_img)
                    labels.append(current_label)
                    image_count += 1

        print(f"   -> {image_count} images générées pour {eleve}")
        current_label += 1

    # --- 4. ENTRAÎNEMENT ---
    if len(faces) > 0:
        print(f"\nEntraînement du modèle sur {len(faces)} visages (Original + Augmenté)...")
        recognizer.train(faces, np.array(labels))
        print("Modèle entraîné et prêt en mémoire !")
        print(f"Mapping : {label_map}")
    else:
        print("ERREUR : Aucune image trouvée.")
else:
    print("ERREUR : Dossier dataset introuvable.")

### Debug

Ici l'objectif est de vérifier que le modèle reconnaît bien des images du dataset_crop et estime un seuil réaliste (LBPH peut facilement être > 100 selon la qualité) pour le live.

In [ ]:
import os
import random
import numpy as np
import cv2

assert 'recognizer' in globals(), "recognizer introuvable: relance la cellule d'entraînement"

DATASET_EVAL_ROOT = LOCAL_CROP_DIR if 'LOCAL_CROP_DIR' in globals() else "/content/dataset_crop"
print("Dataset eval:", DATASET_EVAL_ROOT)

# Dataset WEBCAM: images déjà prétraitées (200x200 GRAY + CLAHE)

def preprocess_for_lbph(img_bgr_or_gray):
    if len(img_bgr_or_gray.shape) == 3:
        g = cv2.cvtColor(img_bgr_or_gray, cv2.COLOR_BGR2GRAY)
    else:
        g = img_bgr_or_gray
    if g.shape != (200, 200):
        g = cv2.resize(g, (200, 200))
    return g

# mapping nom -> id (basé sur label_map courant)
name_to_id = {v: k for k, v in label_map.items()} if 'label_map' in globals() else {}

rows = []  # (expected_name, predicted_name, distance)

for name in sorted(os.listdir(DATASET_EVAL_ROOT)):
    p = os.path.join(DATASET_EVAL_ROOT, name)
    if not os.path.isdir(p):
        continue
    files = [f for f in os.listdir(p) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    if not files:
        continue

    # on échantillonne pour aller vite
    sample = random.sample(files, k=min(12, len(files)))

    for f in sample:
        img = cv2.imread(os.path.join(p, f))
        if img is None:
            continue
        face = preprocess_for_lbph(img)

        pred_id, dist = recognizer.predict(face)
        pred_name = label_map.get(pred_id, "<?>")
        rows.append((name, pred_name, float(dist)))

# Résultats
if not rows:
    print("Aucun échantillon lu. Vérifie DATASET_EVAL_ROOT.")
else:
    total = len(rows)
    correct = sum(1 for exp, pred, _ in rows if exp == pred)
    acc = correct / total

    dists_correct = [d for exp, pred, d in rows if exp == pred]
    dists_wrong = [d for exp, pred, d in rows if exp != pred]

    print(f"Samples: {total} | Accuracy (sur samples du dataset_crop): {acc*100:.1f}%")

    if dists_correct:
        q50 = float(np.quantile(dists_correct, 0.50))
        q90 = float(np.quantile(dists_correct, 0.90))
        q95 = float(np.quantile(dists_correct, 0.95))
        mx = float(np.max(dists_correct))
        print("Distances CORRECTES:")
        print(f"- median={q50:.1f} | p90={q90:.1f} | p95={q95:.1f} | max={mx:.1f}")
        print(f"Seuil conseillé (p95 + marge): {min(300, q95 + 10):.1f}")
    else:
        print("Aucune prédiction correcte sur le dataset_crop -> problème de modèle/mapping/prétraitement.")

    if dists_wrong:
        mnw = float(np.min(dists_wrong))
        q10w = float(np.quantile(dists_wrong, 0.10))
        print("Distances FAUSSES (plus bas = plus dangereux):")
        print(f"- min={mnw:.1f} | p10={q10w:.1f}")

    # affiche quelques erreurs typiques
    errors = [(e, p, d) for e, p, d in rows if e != p]
    errors = sorted(errors, key=lambda t: t[2])[:10]
    if errors:
        print("\nErreurs (les plus 'confiantes'):")
        for e, p, d in errors:
            print(f"- expected={e} predicted={p} dist={d:.1f}")



En plus du debug sur le dataset, on fais un premier petit test live pour juger du seuil optimal

In [ ]:
import time
import numpy as np
import base64
import cv2
from IPython.display import Javascript, display
from google.colab.output import eval_js

assert 'recognizer' in globals(), "Lance l'entraînement d'abord"
assert 'label_map' in globals(), "label_map introuvable"

# Injecte / ré-injecte la fonction JS stream_frame
js_camera_calib = """
    if (!document.getElementById('videoElement')) {
        var video = document.createElement('video');
        video.id = 'videoElement';
        video.style.display = 'none';
        document.body.appendChild(video);
        var canvas = document.createElement('canvas');
        canvas.id = 'canvasElement';
        canvas.style.display = 'none';
        document.body.appendChild(canvas);
    }
    async function stream_frame() {
        var video = document.getElementById('videoElement');
        var canvas = document.getElementById('canvasElement');
        if (video.paused) {
            var stream = await navigator.mediaDevices.getUserMedia({
              video: {
                width: {ideal: 1280},
                height: {ideal: 720},
                facingMode: 'user'
              }
            });
            video.srcObject = stream;
            await video.play();
        }
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        return canvas.toDataURL('image/jpeg', 0.92);
    }
"""
display(Javascript(js_camera_calib))

def js_to_image(js_reply):
    image_bytes = base64.b64decode(js_reply.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

# Détecteur + CLAHE (comme le live)
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

N = 30
print(f"Calibration live: capture {N} frames (mets-toi comme pendant l'appel)")

dists = []
tries = 0
while len(dists) < N and tries < N * 15:
    tries += 1
    js_reply = eval_js('stream_frame()')
    frame = js_to_image(js_reply)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(140, 140))
    if len(faces) == 0:
        continue

    (x, y, w, h) = max(faces, key=lambda f: f[2] * f[3])
    margin = max(10, int(0.15 * min(w, h)))
    x0 = max(0, x - margin)
    y0 = max(0, y - margin)
    x1 = min(gray.shape[1], x + w + margin)
    y1 = min(gray.shape[0], y + h + margin)

    face_roi = gray[y0:y1, x0:x1]
    face_resized = cv2.resize(face_roi, (200, 200))
    face_ready = clahe.apply(face_resized)

    _, dist = recognizer.predict(face_ready)
    dists.append(float(dist))
    time.sleep(0.05)

if not dists:
    print("Aucun visage détecté pendant la calibration")
else:
    d = np.array(dists)
    p95 = float(np.quantile(d, 0.95))
    reco = p95 + 10
    print(f"{len(dists)} samples | dist median={np.median(d):.1f} p90={np.quantile(d,0.9):.1f} p95={p95:.1f} max={np.max(d):.1f}")
    print(f"Reco: mets SEUIL_DISTANCE ≈ p95 + 10 => {reco:.1f}")



## 4. Pipeline SPOT

### Importation des bibliothèques

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Javascript, clear_output
from google.colab.output import eval_js
import base64
import cv2
import numpy as np
import PIL.Image
import os
import time
from datetime import datetime

### Configuration des variables

In [ ]:
# Configuration des variables
BUFFER_TIME = 5.0

### Chargement du modèle

In [ ]:
import os
import cv2

# On force un dataset de référence cohérent avec l'entraînement (crop visage)
TRAIN_DATASET_ROOT = LOCAL_CROP_DIR if 'LOCAL_CROP_DIR' in globals() else "/content/dataset_crop"

# On vérifie si le 'recognizer' existe déjà en mémoire (si tu viens de lancer l'étape 3)
if 'recognizer' in globals():
    print("Modèle détecté en mémoire (venant de l'entraînement). Pas besoin de recharger le fichier.")
else:
    # Cas où tu redémarres le notebook le lendemain
    print("Modèle non trouvé en mémoire. Tentative de chargement depuis le disque...")
    if os.path.exists("spot_lbph_model.yml"):
        recognizer = cv2.face.LBPHFaceRecognizer_create(
            radius=3,
            neighbors=12,
            grid_x=10,
            grid_y=10
        )
        recognizer.read("spot_lbph_model.yml")
        print("Modèle chargé depuis le fichier 'spot_lbph_model.yml'.")
    else:
        print("ERREUR CRITIQUE : Le fichier modèle n'existe pas. Relance l'Étape 3 (Entraînement) !")

# Mapping des labels (doit matcher le dataset utilisé pour l'entraînement)
if os.path.exists(TRAIN_DATASET_ROOT):
    label_map = {i: name for i, name in enumerate(sorted(os.listdir(TRAIN_DATASET_ROOT)))}
    print(f"Classes chargées : {label_map}")
else:
    print(f"Dataset de référence introuvable : {TRAIN_DATASET_ROOT} (le mapping peut être faux)")

# Seuil unique (LBPH: plus bas = plus strict)
SEUIL_DISTANCE = 175
print(f"Seuil de distance configuré à : {SEUIL_DISTANCE}")


### Interface et lancement du cours

C'est ici que l'on voit le résultat, il faut se placer devant la caméra après avoir entrainé le modèle.

Il va alors détecter les personnes présente, absente etc.

Chaque personne est décompté en temps.

Les données sont ensuite stocké pour envoyer au rapport du cours.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Javascript, clear_output
from google.colab.output import eval_js
import base64
import cv2
import numpy as np
import os
import time
from datetime import datetime

# --- CONFIGURATION ---
BUFFER_TIME = 5.0

# CLAHE (doit matcher la capture dataset WEBCAM)
# - Le dataset WEBCAM est enregistré en (200x200 GRAY + CLAHE)
# - Donc en live, on applique CLAHE 1 fois sur la frame brute pour obtenir la même représentation.
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

def predict_lbph_stable(face_ready_gray_200):
    """Prédiction LBPH plus robuste: vote sur variantes + médiane des distances.

    Retourne: (name: str, distance: float, is_match: bool)
    """
    variants = [
        face_ready_gray_200,
        cv2.flip(face_ready_gray_200, 1),
        cv2.GaussianBlur(face_ready_gray_200, (3, 3), 0),
    ]

    preds = []
    for v in variants:
        label_id, dist = recognizer.predict(v)
        preds.append((label_id, float(dist)))

    # vote majoritaire
    counts = {}
    for lid, _ in preds:
        counts[lid] = counts.get(lid, 0) + 1
    best_label_id = max(counts.items(), key=lambda kv: kv[1])[0]

    # distance robuste (médiane des distances du label gagnant)
    dists = sorted([d for lid, d in preds if lid == best_label_id])
    median_dist = dists[len(dists)//2]

    name = label_map.get(best_label_id, "Inconnu")

    # IMPORTANT: ici on renvoie uniquement la stabilité du vote (2/3), PAS le seuil.
    # Le seuil est géré plus bas (adaptatif selon taille du visage).
    vote_ok = (counts[best_label_id] >= 2)
    return name, median_dist, vote_ok

# --- CAMÉRA JS ---
js_camera = """
    if (!document.getElementById('videoElement')) {
        var video = document.createElement('video');
        video.id = 'videoElement';
        video.style.display = 'none';
        document.body.appendChild(video);
        var canvas = document.createElement('canvas');
        canvas.id = 'canvasElement';
        canvas.style.display = 'none';
        document.body.appendChild(canvas);
    }
    async function stream_frame() {
        var video = document.getElementById('videoElement');
        var canvas = document.getElementById('canvasElement');
        if (video.paused) {
            // Plus de pixels + meilleure compression => LBPH beaucoup plus stable
            var stream = await navigator.mediaDevices.getUserMedia({
              video: {
                width: {ideal: 1280},
                height: {ideal: 720},
                facingMode: 'user'
              }
            });
            video.srcObject = stream;
            await video.play();
        }
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        return canvas.toDataURL('image/jpeg', 0.92);
    }
"""
display(Javascript(js_camera))

def js_to_image(js_reply):
    image_bytes = base64.b64decode(js_reply.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

def format_time(seconds):
    m, s = divmod(int(seconds), 60)
    return f"{m:02d}:{s:02d}"

# --- INITIALISATION SESSION ---
# (On s'assure que ELEVES_ATTENDUS existe, sinon liste vide pour test)
if 'ELEVES_ATTENDUS' not in globals(): ELEVES_ATTENDUS = []

global db_eleves_session, start_time_session, end_time_session
db_eleves_session = {
    eleve: {'first_seen': None, 'last_seen': None, 'status': 'ABSENT'}
    for eleve in ELEVES_ATTENDUS
}
start_time_session = time.time()
end_time_session = None

# Chargement du classifieur Haar
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

# --- UI ---
header_widget = widgets.HTML("<h2>Initialisation...</h2>")
video_widget = widgets.Image(format='jpeg', width=500, height=375)
status_widget = widgets.HTML(value="", layout=widgets.Layout(width='400px', height='375px', border='1px solid #ccc', overflow='auto', padding='5px'))

print(f"COURS DÉMARRÉ. Seuil Distance : {SEUIL_DISTANCE}")
display(header_widget)
display(widgets.HBox([video_widget, status_widget]))

# Seuils: en pratique, quand 2 personnes sont dans le cadre, chaque visage est plus petit
# => distances LBPH montent (tu observes ~190 solo, ~200-220 à 2). On adapte le seuil.
SEUIL_DISTANCE_BIG_FACE = 205    # visage grand (solo / proche)
SEUIL_DISTANCE_SMALL_FACE = 235  # visage plus petit
SEUIL_DISTANCE_MULTI_FACE = 245  # quand 2+ visages sont détectés dans la frame
MIN_BIG_FACE = 340              # px (min(w,h))

# Anti-flap: exige X frames consécutives pour valider un élève
CONSECUTIVE_MATCHES_REQUIRED = 3
CONSECUTIVE_MATCHES_REQUIRED_SMALL_FACE = 3
match_streak = {}

# --- BOUCLE PRINCIPALE ---
try:
    while True:
        current_time = time.time()
        duree_cours = current_time - start_time_session
        header_widget.value = f"<h2 style='color:#333;'>DURÉE DU COURS : {format_time(duree_cours)}</h2>"

        # 1. Capture Image
        js_reply = eval_js('stream_frame()')
        frame = js_to_image(js_reply)

        # 2. Conversion Gris + Détection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # IMPORTANT: on veut un visage assez grand et on garde uniquement le + grand
        # (sinon LBPH décroche et donne des distances 150-300)
        faces = face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.2,
            minNeighbors=5,
            minSize=(140, 140)
        )

        if len(faces) > 0:
            faces = sorted(faces, key=lambda f: f[2] * f[3], reverse=True)

        num_faces = len(faces)
        display_frame = frame.copy()

        for (x, y, w, h) in faces:
            # 3. Préparation Visage pour LBPH (mêmes idées que le crop dataset)
            margin = max(10, int(0.15 * min(w, h)))
            x0 = max(0, x - margin)
            y0 = max(0, y - margin)
            x1 = min(gray.shape[1], x + w + margin)
            y1 = min(gray.shape[0], y + h + margin)

            face_roi = gray[y0:y1, x0:x1]

            face_resized = cv2.resize(face_roi, (200, 200))
            # IMPORTANT: CLAHE 1x (pour matcher le dataset WEBCAM)
            face_ready = clahe.apply(face_resized)

            # 4. Prédiction (plus robuste)
            try:
                # Seuil adaptatif (2 personnes => visages plus petits => distances plus hautes)
                is_big_face = min(w, h) >= MIN_BIG_FACE
                seuil = SEUIL_DISTANCE_BIG_FACE if is_big_face else SEUIL_DISTANCE_SMALL_FACE
                if num_faces >= 2:
                    seuil = max(seuil, SEUIL_DISTANCE_MULTI_FACE)

                required = CONSECUTIVE_MATCHES_REQUIRED if is_big_face else CONSECUTIVE_MATCHES_REQUIRED_SMALL_FACE

                # Vote: sur petits visages / multi-face, le vote flip/blur peut être instable.
                # Donc on accepte le vote même si ce n'est pas 2/3 (vote_ok=1) quand 2+ visages.
                detected_name, distance, vote_ok = predict_lbph_stable(face_ready)
                if num_faces >= 2:
                    vote_ok = True

                is_match = vote_ok and (distance < seuil)

                # Anti-flap: streak par nom (suffisant pour un système d'appel)

                if is_match:
                    match_streak[detected_name] = match_streak.get(detected_name, 0) + 1
                else:
                    match_streak[detected_name] = max(0, match_streak.get(detected_name, 0) - 1)

                stable_match = is_match and (match_streak.get(detected_name, 0) >= required)

                if stable_match:
                    # correspondance valide (stable)
                    if detected_name in db_eleves_session:
                        color = (0, 255, 0)  # Vert (Présent)
                        label_text = f"{detected_name} ({int(distance)}) {w}x{h}"

                        # Update DB
                        db_eleves_session[detected_name]['last_seen'] = current_time
                        if db_eleves_session[detected_name]['first_seen'] is None:
                            db_eleves_session[detected_name]['first_seen'] = current_time
                    else:
                        color = (0, 165, 255)  # Orange (Intrus connu)
                        label_text = f"Intrus: {detected_name} ({int(distance)})"
                else:
                    # Trop de distance ou pas stable -> Inconnu
                    color = (0, 0, 255)  # Rouge
                    # debug: meilleur label prédit même si on rejette + taille visage
                    label_text = f"Inconnu ({int(distance)}) best:{detected_name} {w}x{h} seuil:{int(seuil)} votes:{int(vote_ok)} streak:{match_streak.get(detected_name, 0)}/{required}"

            except Exception:
                label_text = "Erreur"
                color = (0, 0, 255)

            # Dessin
            cv2.rectangle(display_frame, (x, y), (x+w, y+h), color, 2)
            cv2.rectangle(display_frame, (x, y-30), (x+w, y), color, -1)
            cv2.putText(display_frame, label_text, (x+5, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

        # --- TABLEAU DE BORD ---
        html_content = "<table style='width:100%; font-family:sans-serif; font-size:13px; border-collapse: collapse;'>"
        html_content += "<tr style='background:#333; color:white;'><th style='padding:5px;'>Élève</th><th>Statut</th><th>Détails</th></tr>"

        for eleve in ELEVES_ATTENDUS:
            data = db_eleves_session[eleve]
            if data['first_seen'] is None:
                status_txt = "ABSENT"
                style = "color:red;"
                retard = current_time - start_time_session
                timer_txt = f"{format_time(retard)}"
            else:
                time_since_last = current_time - data['last_seen']
                retard_arrivee = data['first_seen'] - start_time_session
                str_arrivee = f"Arr: +{format_time(retard_arrivee)}"

                if time_since_last < BUFFER_TIME:
                    status_txt = "PRÉSENT"
                    style = "color:green; font-weight:bold;"
                    timer_txt = str_arrivee
                else:
                    status_txt = "PARTI"
                    style = "color:orange; font-weight:bold;"
                    timer_txt = f"Parti: {format_time(time_since_last)}"

            html_content += f"<tr style='border-bottom:1px solid #ddd; {style}'>"
            html_content += f"<td style='padding:5px;'>{eleve}</td>"
            html_content += f"<td style='text-align:center;'>{status_txt}</td>"
            html_content += f"<td style='text-align:right; font-family:monospace;'>{timer_txt}</td></tr>"

        html_content += "</table>"

        _, encoded_img = cv2.imencode('.jpg', display_frame)
        video_widget.value = encoded_img.tobytes()
        status_widget.value = html_content

except KeyboardInterrupt:
    end_time_session = time.time()
    try:
        from zoneinfo import ZoneInfo
        now_local = datetime.now(ZoneInfo("Europe/Paris"))
    except Exception:
        # fallback: heure système (peut rester UTC sur Colab)
        now_local = datetime.now()
    print(f"\n✅ Cours arrêté. Fin : {now_local.strftime('%H:%M:%S')}")
except Exception as e:
    print(f"Erreur Critique : {e}")

## 5. Génération du rapport final du cours

Une fois le cours fini, toutes les variables sont stockées, on fait les calculs pour le retard, la sortie anticipé et l'absence.
On affiche le compte rendu du cours

In [ ]:
from datetime import datetime
import pandas as pd
import time

# Vérification de sécurité (si on lance cette cellule sans avoir fait le cours)
if 'end_time_session' not in globals() or end_time_session is None:
    print("Attention : Le cours n'a pas été arrêté correctement ou n'a pas commencé.")
    end_time_session = time.time() # Valeur par défaut pour ne pas planter

# Calcul sur la base du temps FIGÉ
duree_totale_cours = end_time_session - start_time_session

print("\n" + "="*80)
print(f"RAPPORT DÉTAILLÉ SPOT - Fin du cours enregistrée")
print(f"⏱Durée VALIDÉE du cours : {format_time(duree_totale_cours)}")
print("="*80)

recap_data = []

for eleve in ELEVES_ATTENDUS:
    data = db_eleves_session[eleve]

    retard_sec = 0.0
    sortie_sec = 0.0
    total_absence_sec = 0.0

    col_retard = "-"
    col_sortie = "-"
    col_total = "-"
    statut_final = "ABSENT"

    # --- CALCULS ---
    if data['first_seen'] is None:
        # ABSENT TOUT LE LONG
        statut_final = "ABSENT"
        total_absence_sec = duree_totale_cours
        col_total = format_time(total_absence_sec)
        col_retard = "Absent tout le cours"
        col_sortie = "-"

    else:
        # VENU AU MOINS UNE FOIS

        # 1. Retard Début
        retard_sec = data['first_seen'] - start_time_session
        if retard_sec > 1.0:
            col_retard = f"+ {format_time(retard_sec)}"
        else:
            col_retard = "À l'heure"
            retard_sec = 0

        # 2. Sortie Fin (Basé sur le temps figé end_time_session)
        temps_depuis_derniere_vue = end_time_session - data['last_seen']

        # Marge technique de 10s
        if temps_depuis_derniere_vue < (BUFFER_TIME + 10.0):
            statut_final = "PRÉSENT"
            col_sortie = "Non (Présent)"
            sortie_sec = 0
        else:
            statut_final = "PARTI AVANT LA FIN"
            sortie_sec = temps_depuis_derniere_vue
            col_sortie = f"- {format_time(sortie_sec)}"

        # 3. Total
        total_absence_sec = retard_sec + sortie_sec
        col_total = format_time(total_absence_sec)

    recap_data.append([eleve, statut_final, col_retard, col_sortie, col_total])

df = pd.DataFrame(recap_data, columns=[
    "Élève",
    "Statut Final",
    "Retard (Début)",
    "Sortie Anticipée (Fin)",
    "Total Absence"
])

# Styles
def color_status(val):
    if val == 'PRÉSENT': return 'color: green; font-weight: bold'
    if val == 'ABSENT': return 'color: red; font-weight: bold'
    if val == 'PARTI AVANT LA FIN': return 'color: orange; font-weight: bold'
    return ''

styles = [
    dict(selector="th", props=[("text-align", "center")]),
    dict(selector="td", props=[("text-align", "center")])
]

display(df.style.map(color_status, subset=['Statut Final']).set_table_styles(styles))